In [1]:
import os
from copy import deepcopy

from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from Environment import Environment, make_one_hot, give_mapping
from Networks import UserActor, AsstActor, CentralizedCritic

In [2]:
def give_prev_steps(prev_steps_assist, steps):
    prev_steps_assist = [[0,0,0,0,-1,-1] for i in range(steps-1)]
    return prev_steps_assist

In [3]:
class Agent:
    def __init__(self):
        self.user_actor = UserActor()
        self.asst_actor = AsstActor()
        self.critic = CentralizedCritic()
        self.optimizer_actors = tf.keras.optimizers.Adam(lr = 0.0001)
        self.optimizer_critic = tf.keras.optimizers.Adam(lr = 0.0002)
        self.huber_loss = tf.keras.losses.Huber()
        self.memory_len = 4
        
        self.gamma = 0.90
        self.env = Environment()
        self.env.cells = np.array([[0.7, 0.1], [0.1, 0.1], [0.5, 0.7], [0.6, 0.2], [0.7, 0.4], [0.2, 0.9]])
        self.env_cell_mapping = give_mapping(self.env.cells)
        self.env_cell_mapping = self.env_cell_mapping[np.newaxis, :, :, np.newaxis]
        self.eps = 10e-6
    
    def learn(self):
        env = self.env
        max_steps = 40
        running_reward = 0
        reached = 0
        best = 0
        
        for epoch in tqdm(range(10000)):
            
            user_action_probs_history = []
            asst_action_probs_history = []
            critic_value_history = []
            rewards_history = []
            returns = [] #Returns
            
            done = False
            episode_reward = 0
            start, dest = env.give_start_dest()
            ob_user = [start[0], start[1], dest[0], dest[1]]
            prev_steps_assist = []
            prev_steps_assist = give_prev_steps(prev_steps_assist, self.memory_len)
            step = 0
            episode_reward = 0
            
            with tf.GradientTape(persistent = True) as tape:
                while not done and step<max_steps:
                    curr_loc = ob_user[:2]
                    target_loc = ob_user[2:4]
                    step+=1
                    ob_user = np.array(ob_user)[np.newaxis]
                    user_probs = self.user_actor.model(ob_user)
                    user_action = np.random.choice(4, p=np.squeeze(user_probs))
                    user_action_probs_history.append(tf.math.log(user_probs[0, user_action]))

                    action_user_one_hot = make_one_hot(user_action, 4)

                    ob_assist = [action_user_one_hot + curr_loc] 
                    ob_assist = prev_steps_assist + ob_assist
                    ob_assist = np.array(ob_assist)[np.newaxis]
                    
                    asst_probs = self.asst_actor.model([ob_assist, self.env_cell_mapping])
                    asst_action = np.random.choice(4, p=np.squeeze(asst_probs))
                    asst_action_probs_history.append(tf.math.log(asst_probs[0, asst_action]))
                    
                    asst_output_one_hot = np.array(make_one_hot(asst_action, 4))[np.newaxis]
                    
                    critic_value = self.critic.model([ob_user, ob_assist, self.env_cell_mapping, asst_output_one_hot])
                    critic_value_history.append(critic_value)
                    
                    new_loc, reward_user, reward_assist, done = self.env.step(user_action, asst_action + 1, target_loc, curr_loc)
                    
                    next_ob_user = new_loc[:]
                    next_ob_user = next_ob_user + target_loc

                    ob_user = next_ob_user
                    prev_steps_assist = np.squeeze(ob_assist).tolist()[1:]
                    
                    rewards_history.append(reward_user)
                    episode_reward+=reward_user
                    
                    if done:
                        reached += 1

                running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

                discounted_sum = 0
                for r in rewards_history[::-1]:
                    discounted_sum = r + self.gamma * discounted_sum
                    returns.append(discounted_sum)
                returns.reverse()
                
                critic_losses = []
                user_losses = []
                asst_losses = []
                
                for log_prob_user, log_prob_asst, val, ret in zip(user_action_probs_history, asst_action_probs_history, critic_value_history,\
                                                                 returns):
                    diff = ret - val
                    user_losses.append(-log_prob_user*diff)
                    asst_losses.append(-log_prob_asst*diff)
                    critic_losses.append(self.huber_loss(tf.expand_dims(val, 0), tf.expand_dims(ret, 0)))

                user_loss = sum(user_losses)
                asst_loss = sum(asst_losses)
                critic_loss = sum(critic_losses)

            grads = tape.gradient(user_loss, self.user_actor.model.trainable_variables)
            self.optimizer_actors.apply_gradients(zip(grads, self.user_actor.model.trainable_variables))

            grads = tape.gradient(asst_loss, self.asst_actor.model.trainable_variables)
            self.optimizer_actors.apply_gradients(zip(grads, self.asst_actor.model.trainable_variables))

            grads = tape.gradient(critic_loss, self.critic.model.trainable_variables)
            self.optimizer_critic.apply_gradients(zip(grads, self.critic.model.trainable_variables))
            
            if epoch%100 == 0:
                print(running_reward)
            
            if epoch and epoch%100 == 0:
                print(reached)
                if reached>=best:
                    best = reached
                    tf.keras.models.save_model(self.user_actor.model, 'user.h5')
                    tf.keras.models.save_model(self.asst_actor.model, 'asst.h5')
                    tf.keras.models.save_model(self.critic.model, 'critic.h5')
                reached = 0
                
                if epoch%200:
                    self.trial()
                
                
    def trial(self):
        env = self.env
        max_steps = 20
        done = False
        episode_reward = 0
        start, dest = env.give_start_dest()
        ob_user = [start[0], start[1], dest[0], dest[1]]
        prev_steps_assist = []
        prev_steps_assist = give_prev_steps(prev_steps_assist, self.memory_len)
        step = 0
        episode_reward = 0

        while not done and step<max_steps:
            curr_loc = ob_user[:2]
            target_loc = ob_user[2:4]
            step+=1
            print(ob_user)

            ob_user = np.array(ob_user)[np.newaxis]
            user_probs = self.user_actor.model(ob_user)
            user_action = np.argmax(np.squeeze(user_probs))

            action_user_one_hot = make_one_hot(user_action, 4)

            ob_assist = [action_user_one_hot + curr_loc] 
            ob_assist = prev_steps_assist + ob_assist
            ob_assist = np.array(ob_assist)[np.newaxis]

            asst_probs = self.asst_actor.model([ob_assist, self.env_cell_mapping])
            print(asst_probs)
            asst_action = np.argmax(np.squeeze(asst_probs)) 
            new_loc, reward_user, reward_assist, done = self.env.step(user_action, asst_action + 1, target_loc, curr_loc)

            next_ob_user = new_loc[:]
            next_ob_user = next_ob_user + target_loc

            ob_user = next_ob_user
            prev_steps_assist = np.squeeze(ob_assist).tolist()[1:]
            episode_reward+=reward_user




In [4]:
agent = Agent()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
subtract (Subtract)             (None, 2)            0           tf_op_layer_strided_slice[0][0]  
                                                                 tf_op_layer_strided_sl

In [5]:
agent.learn()

  0%|                                                                             | 1/10000 [00:04<13:40:42,  4.92s/it]

-2.0


  1%|▊                                                                           | 100/10000 [03:19<6:21:29,  2.31s/it]

-37.49327097516518
15
[0.8, 0.0, 0.6, 0.2]
tf.Tensor([[0.18986392 0.29892808 0.2663672  0.24484079]], shape=(1, 4), dtype=float32)
[1.0, 0.0, 0.6, 0.2]
tf.Tensor([[0.18842435 0.2995277  0.25923106 0.2528169 ]], shape=(1, 4), dtype=float32)
[1.0, 0.0, 0.6, 0.2]
tf.Tensor([[0.1873081  0.30114833 0.25204197 0.25950158]], shape=(1, 4), dtype=float32)
[1.0, 0.0, 0.6, 0.2]
tf.Tensor([[0.18759032 0.30375895 0.24658625 0.26206455]], shape=(1, 4), dtype=float32)
[1.0, 0.0, 0.6, 0.2]
tf.Tensor([[0.19079363 0.30227187 0.24208172 0.26485276]], shape=(1, 4), dtype=float32)
[1.0, 0.0, 0.6, 0.2]
tf.Tensor([[0.19079363 0.30227187 0.24208172 0.26485276]], shape=(1, 4), dtype=float32)
[1.0, 0.0, 0.6, 0.2]
tf.Tensor([[0.19079363 0.30227187 0.24208172 0.26485276]], shape=(1, 4), dtype=float32)
[1.0, 0.0, 0.6, 0.2]
tf.Tensor([[0.19079363 0.30227187 0.24208172 0.26485276]], shape=(1, 4), dtype=float32)
[1.0, 0.0, 0.6, 0.2]
tf.Tensor([[0.19079363 0.30227187 0.24208172 0.26485276]], shape=(1, 4), dtype=float3

  1%|▊                                                                           | 101/10000 [03:21<6:39:07,  2.42s/it]

...

 99%|██████████████████████████████████████████████████████████████████████████▎| 9900/10000 [3:35:47<01:03,  1.58it/s]

-4.017943176209828
94
[0.6, 0.7, 0.6, 0.2]
tf.Tensor([[0.00528828 0.17147344 0.7860138  0.03722442]], shape=(1, 4), dtype=float32)
[0.6, 0.4, 0.6, 0.2]
tf.Tensor([[0.00639358 0.18035056 0.77308106 0.04017481]], shape=(1, 4), dtype=float32)
[0.6, 0.1, 0.6, 0.2]
tf.Tensor([[0.01326304 0.21159863 0.7242547  0.05088358]], shape=(1, 4), dtype=float32)
[0.6, 0.4, 0.6, 0.2]
tf.Tensor([[0.1278641  0.3461463  0.4854215  0.04056813]], shape=(1, 4), dtype=float32)
[0.6, 0.1, 0.6, 0.2]
tf.Tensor([[0.14036207 0.34577325 0.46725863 0.04660609]], shape=(1, 4), dtype=float32)
[0.6, 0.4, 0.6, 0.2]
tf.Tensor([[0.11982757 0.34137014 0.4935509  0.0452515 ]], shape=(1, 4), dtype=float32)
[0.6, 0.1, 0.6, 0.2]
tf.Tensor([[0.14036207 0.34577325 0.46725863 0.04660609]], shape=(1, 4), dtype=float32)
[0.6, 0.4, 0.6, 0.2]
tf.Tensor([[0.11982757 0.34137014 0.4935509  0.0452515 ]], shape=(1, 4), dtype=float32)
[0.6, 0.1, 0.6, 0.2]
tf.Tensor([[0.14036207 0.34577325 0.46725863 0.04660609]], shape=(1, 4), dtype=float3

 99%|██████████████████████████████████████████████████████████████████████████▎| 9901/10000 [3:35:48<01:00,  1.65it/s]


[0.6, 0.4, 0.6, 0.2]
tf.Tensor([[0.11982757 0.34137014 0.4935509  0.0452515 ]], shape=(1, 4), dtype=float32)
[0.6, 0.1, 0.6, 0.2]
tf.Tensor([[0.14036207 0.34577325 0.46725863 0.04660609]], shape=(1, 4), dtype=float32)
[0.6, 0.4, 0.6, 0.2]
tf.Tensor([[0.11982757 0.34137014 0.4935509  0.0452515 ]], shape=(1, 4), dtype=float32)


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [3:37:20<00:00,  1.30s/it]


In [6]:
agent.trial()

[0.3, 0.2, 0.2, 0.9]
tf.Tensor([[0.00582913 0.17895536 0.7793471  0.03586836]], shape=(1, 4), dtype=float32)
[0.3, 0.5, 0.2, 0.9]
tf.Tensor([[0.00809922 0.19222458 0.7576188  0.04205743]], shape=(1, 4), dtype=float32)
[0.3, 0.8, 0.2, 0.9]
tf.Tensor([[0.0160022  0.2217195  0.7092075  0.05307089]], shape=(1, 4), dtype=float32)
[0.0, 0.8, 0.2, 0.9]
tf.Tensor([[0.0756771  0.30847886 0.57068694 0.04515709]], shape=(1, 4), dtype=float32)
[0.3, 0.8, 0.2, 0.9]
tf.Tensor([[0.10498974 0.32770896 0.5189384  0.04836291]], shape=(1, 4), dtype=float32)
[0.0, 0.8, 0.2, 0.9]
tf.Tensor([[0.11071416 0.3404957  0.5080102  0.04077994]], shape=(1, 4), dtype=float32)
[0.3, 0.8, 0.2, 0.9]
tf.Tensor([[0.11487425 0.33421937 0.5067677  0.04413863]], shape=(1, 4), dtype=float32)
[0.0, 0.8, 0.2, 0.9]
tf.Tensor([[0.11071416 0.3404957  0.5080102  0.04077994]], shape=(1, 4), dtype=float32)
[0.3, 0.8, 0.2, 0.9]
tf.Tensor([[0.11487425 0.33421937 0.5067677  0.04413863]], shape=(1, 4), dtype=float32)
[0.0, 0.8, 0.2, 0.9